In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

In [3]:
import numpy as np
import os
import time

### Read the data

In [4]:
text_path = "/content/drive/MyDrive/Colab Notebooks/NLP/2/10-Text Generator(Attar's Poem)/Sample/naserkhosro.txt"
text = open(text_path, 'rb').read().decode(encoding='utf-8')

In [5]:
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 633429 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

ای قبهٔ گردندهٔ بی روزن خضرا
با قامت فرتوتی و با قوت برنا
فرزند توایم ای فلک، ای مادر بدمهر
ای مادر ما چونکه همی کین کشی از ما؟
فرزند تو این تیره تن خامش خاکی است
پاکیزه خرد نیست نه این جوهر گویا
تن خانهٔ این گوهر والای شریف است
تو مادر این خانهٔ این


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

50 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [9]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '(' :   3,
  ')' :   4,
  '.' :   5,
  ':' :   6,
  '«' :   7,
  '»' :   8,
  '،' :   9,
  '؛' :  10,
  '؟' :  11,
  'ء' :  12,
  'آ' :  13,
  'ؤ' :  14,
  'ئ' :  15,
  'ا' :  16,
  'ب' :  17,
  'ة' :  18,
  'ت' :  19,
  ...
}


In [10]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:30]), text_as_int[:30]))

'ای قبهٔ گردندهٔ بی روزن خضرا\nب' ---- characters mapped to int ---- > [16 49  1 37 17 41 43  1 48 26 24 40 24 41 43  1 17 49  1 26 42 27 40  1
 23 31 26 16  0 17]


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [11]:
# The maximum length sentence we want for a single input in characters
seq_length = 100

In [12]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [13]:
for i in char_dataset.take(10):
    print(i.numpy(), idx2char[i.numpy()])

16 ا
49 ی
1  
37 ق
17 ب
41 ه
43 ٔ
1  
48 گ
26 ر


### Sliding Window for Sequence Generation

Instead of using simple batching, we use the **sliding window** method to generate overlapping sequences of length `seq_length + 1`.  
This ensures that every character in the text contributes to multiple training samples, preserving the continuity of the text.  
Compared to fixed-size batching, this approach creates **richer and more context-aware training data**, helping the model better capture character-level dependencies in the text.


In [14]:
# Sliding window: ساخت پنجره‌های طول ثابت با گام 1
sequences = char_dataset.window(size=seq_length + 1, shift=1, drop_remainder=True)

In [15]:
# هر پنجره را به یک تانسور تبدیل می‌کنیم تا بتوان پردازشش کرد
sequences = sequences.flat_map(lambda window: window.batch(seq_length + 1))

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [16]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [17]:
# بررسی ۳ نمونه اولیه
for input_example, target_example in dataset.take(3):
    print('Input: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target:', repr(''.join(idx2char[target_example.numpy()])))
    print('-' * 30)

Input:  'ای قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر '
Target: 'ی قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر م'
------------------------------
Input:  'ی قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر م'
Target: ' قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر ما'
------------------------------
Input:  ' قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر ما'
Target: 'قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر ما '
------------------------------


### Create Training Batches

After generating overlapping input-target pairs using the **sliding window**, we now prepare the data for efficient training.

Using `tf.data.Dataset.batch`, we group these pairs into batches of size `BATCH_SIZE`.  
This batching ensures optimized parallel processing on GPUs and uniform batch sizes by setting `drop_remainder=True`.  
The final `dataset` contains batches of sequences, each ready to be fed into the model during training.


In [18]:
BATCH_SIZE = 32
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(32, None), dtype=tf.int64, name=None), TensorSpec(shape=(32, None), dtype=tf.int64, name=None))>

## Build the Model

We define the model using `tf.keras.Sequential`, composed of three main layers:

- **Embedding Layer**: Maps character indices into dense vectors of size `embedding_dim`. This layer is trainable and learns character-level embeddings.
- **GRU Layer**: A type of recurrent neural network that captures sequence dependencies. We use `return_sequences=True` so the model outputs a prediction at each time step. The `stateful=True` option helps the model retain memory across batches for better long-term dependency learning.
- **Dense Layer**: Outputs logits for each character in the vocabulary, one per time step.

The model is built using the specified `vocab_size`, `embedding_dim`, and `rnn_units`.

### Flexible Model Builder with GRU or LSTM

We enhance the model-building function to support both GRU and LSTM layers, selectable via a `rnn_type` argument.

- `GRU`: Efficient and faster to train
- `LSTM`: Better for capturing long-term dependencies

Dropout is added to reduce overfitting. Layer normalization is included after the RNN layer for better training stability.
This setup allows easy experimentation and comparison between different RNN architectures.


In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size,
                rnn_type='GRU', dropout_rate=0.2):

    inputs = tf.keras.Input(batch_shape=(batch_size, None), dtype=tf.int32)

    x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)

    if rnn_type.upper() == 'GRU':
        x = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            stateful=True,
            dropout=dropout_rate,
            recurrent_initializer='glorot_uniform'
        )(x)
    elif rnn_type.upper() == 'LSTM':
        x = tf.keras.layers.LSTM(
            rnn_units,
            return_sequences=True,
            stateful=True,
            dropout=dropout_rate,
            recurrent_initializer='glorot_uniform'
        )(x)
    else:
        raise ValueError("rnn_type must be either 'GRU' or 'LSTM'")

    x = tf.keras.layers.LayerNormalization()(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)

    model = tf.keras.Model(inputs, outputs)
    return model


In [21]:
model_gru = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    rnn_type='GRU'
)

model_lstm = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    rnn_type='LSTM'
)

### Try the Model

Before training, we can run the model on a sample input batch to verify its behavior.

- The model outputs a tensor of shape `(batch_size, sequence_length, vocab_size)`.
- To simulate actual text generation, we sample from the output distribution at each time step using `tf.random.categorical`, instead of taking `argmax`, to avoid repetitive predictions.
- Finally, we decode the predicted character indices to see the raw output of the untrained model.


In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_gru.predict(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
(32, 100, 50) # (batch_size, sequence_length, vocab_size)


In [23]:
example_batch_predictions

array([[[ 1.62707195e-01, -2.98130155e-01, -2.36787617e-01, ...,
          1.49577856e-01,  3.55823576e-01, -4.00275648e-01],
        [-6.54234365e-02, -2.94892520e-01,  2.56340057e-01, ...,
          1.05396621e-01,  3.34111243e-01, -4.77799624e-01],
        [-3.69009554e-01,  1.51859149e-01, -3.57400328e-01, ...,
          2.31665418e-01,  3.01686674e-01, -6.92625344e-01],
        ...,
        [ 3.09291929e-01, -2.45484009e-01, -2.58091152e-01, ...,
          3.09803396e-01,  6.84053540e-01, -2.84064174e-01],
        [ 1.30623877e-01,  1.70978546e-01, -4.79557753e-01, ...,
          5.66088557e-01,  7.97091722e-01, -4.21520293e-01],
        [-2.22277001e-01,  2.62772322e-01, -7.75019228e-01, ...,
          4.68940049e-01,  4.26169038e-01, -5.74203610e-01]],

       [[ 1.21824197e-01, -6.69976592e-01, -4.68687713e-02, ...,
          3.00965101e-01,  7.75108188e-02, -3.10899794e-01],
        [-2.47942626e-01, -2.46437505e-01, -2.00202703e-01, ...,
          1.20065153e-01,  1.87305763e

In [24]:
model_gru.summary()
print('\n' * 3)
model_lstm.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (32, None)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (32, None, 256)        │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (32, None, 1024)       │     3,938,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (32, None, 1024)       │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, None, 50)         │        51,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,004,402 (15.28 MB)

 Trainable params: 4,004,402 (15.28 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (32, None)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (32, None, 256)        │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (32, None, 1024)       │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (32, None, 1024)       │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, None, 50)         │        51,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,313,074 (20.27 MB)

 Trainable params: 5,313,074 (20.27 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
sampled_indices = tf.random.categorical(
    logits=example_batch_predictions[0],  # shape = (sequence_length, vocab_size)
    num_samples=1
)

In [26]:
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [27]:
sampled_indices

array([ 0, 45, 12,  1,  3, 43, 21, 49, 10, 13, 14,  6, 37,  4, 33, 26,  9,
       41, 46, 26, 34, 31,  7, 39, 16, 32, 14,  5, 42, 35, 44, 27, 19, 23,
       20,  9, 34,  5, 35,  6, 26, 12, 25,  7,  6, 47, 22,  1,  0, 21, 49,
       41, 37,  9, 49, 16, 40, 40, 30, 34, 29, 32, 19, 27, 28, 41, 38,  3,
        9,  2,  2,  0, 38, 44, 37, 20, 25, 34, 19, 16, 25,  8,  4,  6, 18,
       38, 31, 26,  2, 33, 24,  7, 17,  3,  0, 30, 35, 35, 29, 13])

In [28]:
input_text = ''.join([idx2char[i] for i in input_example_batch[0].numpy()])

In [29]:
predicted_text = ''.join([idx2char[i] for i in sampled_indices])

In [30]:
print("Input: \n", repr(input_text))
print()
print("Next Char Predictions: \n", repr(predicted_text))

Input: 
 'ای قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر '

Next Char Predictions: 
 '\nچء (ٔجی؛آؤ:ق)ظر،هژرعض«ماطؤ.وغپزتخث،ع.غ:رءذ«:کح \nجیهق،یاننصعشطتزسهل(،!!\nلپقثذعتاذ»):ةلضر!ظد«ب(\nصغغشآ'


## Train the model

### Loss Function and Model Compilation

We define a custom loss function using `sparse_categorical_crossentropy`, suitable for multi-class classification tasks with integer labels.  
Since the model outputs raw logits (not softmax probabilities), we set `from_logits=True`.

We then compile the model using the Adam optimizer and our loss function to prepare it for training.


In [31]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [32]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)

In [33]:
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Example batch loss (mean):", tf.reduce_mean(example_batch_loss).numpy())

Prediction shape:  (32, 100, 50)  # (batch_size, sequence_length, vocab_size)
Example batch loss (mean): 4.020863


In [35]:
# GRU
model_gru.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# LSTM
model_lstm.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)